In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import time

url = 'https://search.naver.com/search.naver?ssc=tab.blog.all&sm=tab_jum&query=%EB%AA%BD%ED%82%BD+%EC%9D%BC%EC%82%B0%EB%B0%A4%EB%A6%AC%EB%8B%A8%EA%B8%B8%EC%B9%B4%ED%8E%98+%EB%B3%B8%EC%A0%90'


In [43]:
import re
import time
import pandas as pd
from urllib.parse import quote_plus

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup


def extract_korean(text: str) -> str:
    """문자열에서 한글만 추출해서 공백으로 이어붙임."""
    if not text:
        return ""
    result = re.compile('[가-힣]+').findall(text)
    return " ".join(result)


def build_search_url(store_name: str) -> str:
    """가게 이름으로 네이버 블로그 검색 URL 생성."""
    query = quote_plus(store_name)  # 공백, 한글 등 인코딩
    url = f"https://search.naver.com/search.naver?ssc=tab.blog.all&sm=tab_jum&query={query}"
    return url


def crawl_naver_blog_reviews(store_name: str, scroll_count: int = 1) -> pd.DataFrame:
    """
    네이버 블로그에서 특정 가게 이름으로 검색한 뒤
    블로그 글 제목, 링크, 내용, 한글만 추출한 내용을 크롤링해서 DataFrame 반환 + CSV 저장.

    Parameters
    ----------
    store_name : str
        검색할 가게 이름 (예: '몽키 일산 밤리단길카페 본점')
    scroll_count : int
        검색 결과 화면에서 PAGE_DOWN 할 횟수 (기본 1번)

    Returns
    -------
    pd.DataFrame
        titles, links, contents, only_kor_contents 컬럼을 가진 DataFrame
    """

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")
    # 1. 크롬 드라이버 실행
    driver = webdriver.Chrome(options=chrome_options)

    try:
        # 2. 검색 페이지 접속
        search_url = build_search_url(store_name)
        driver.get(search_url)
        time.sleep(3)

        # 3. 검색 결과 페이지 스크롤 (조금 더 로딩되게)
        body = driver.find_element(By.CSS_SELECTOR, 'body')
        for _ in range(scroll_count):
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(2)

        # 4. HTML 가져와서 BeautifulSoup으로 파싱
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 5. 검색 결과에서 블로그 제목/링크 추출 (현재 네이버 구조 기준)
        #'div > div.sds-comps-vertical-layout.sds-comps-full-layout.N8JknjVu2Kc8aLavF6ZA > a'
        url_soup = soup.select('div > div.sds-comps-vertical-layout.sds-comps-full-layout.N8JknjVu2Kc8aLavF6ZA > a') + soup.select('div > div.sds-comps-vertical-layout.sds-comps-full-layout.MkfloTjOr2Rg4LLlLwVA > a')

        titles = []
        links = []

        for t in url_soup:
            title_text = t.get_text().strip()
            link = t.attrs.get('href', '').strip()
            if not link:
                continue

            print(title_text, link)  # 디버깅용 출력
            titles.append(title_text)
            links.append(link)

        # 6. 각 블로그 글에 들어가서 본문 크롤링
        contents = []

        for link in links:
            try:
                driver.get(link)
                time.sleep(2)

                # 새 에디터/구 에디터 섞여 있어서 try-except
                try:
                    # 구버전 블로그(iframe) 구조
                    driver.switch_to.frame("mainFrame")
                except Exception:
                    print('error')
                    # iframe이 없을 수도 있으니 무시
                    pass

                try:
                    text = driver.find_element(By.CSS_SELECTOR, 'div.se-main-container').text
                except Exception:
                    # 다른 구조의 블로그일 경우 대비
                    try:
                        text = driver.find_element(By.CSS_SELECTOR, 'div#postViewArea').text
                    except Exception:
                        text = ""

                contents.append(text)
            except Exception as e:
                print(f"[ERROR] {link} 크롤링 중 오류: {e}")
                contents.append("")

            # 항상 frame 다시 초기화
            driver.switch_to.default_content()

        # 7. 한글만 남긴 컬럼 생성
        only_kor_contents = [extract_korean(c) for c in contents]

        # 8. DataFrame 생성
        review_data = pd.DataFrame({
            'titles': titles,
            'links': links,
            'contents': contents,
            'only_kor_contents': only_kor_contents
        })

        # 9. 파일명에 쓸 수 없는 문자 제거 후 CSV 저장
        safe_store_name = re.sub(r'[\\/*?:"<>|]', "_", store_name)
        filename = f"{safe_store_name}_review_data.csv"
        review_data.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"✅ '{filename}' 파일로 저장 완료")

        return review_data

    finally:
        driver.quit()




NameError: name 'search_url' is not defined

In [44]:
if __name__ == "__main__":
    # 예시 실행
    store = "카피로우 일산밤리단길카페점"
    df = crawl_naver_blog_reviews(store_name=store, scroll_count=1)

디저트가 특히 맛있었던 일산 카페, 카피로우 일산밤리단길카페점 방문기 https://blog.naver.com/hidew777/224076747828
카피로우 일산밤리단길카페점 솔직후기 https://blog.naver.com/rhdmsdl0720/223996105928
카피로우 일산밤리단길카페점 내돈내산 솔직후기 https://blog.naver.com/rkguscjswo26/223891656310
“카피로우” 일산밤리단길카페점.크렘당쥬?프랑스디저트 천사의크림일산 밤리단길카페추천 https://blog.naver.com/comma2024/223785598110
[경기/고양시] 일산동구 마두동 밤리단길 데이트 카페 추천 무화과 디저트 맛집 찾고 있다면? ‘카피로우 일산밤리단길카페점’ - 아메리카노, 무화과요거트케이크(글루텐프리) https://blog.naver.com/vivid_07/224064134098
일산 카페 찐맛집, 카피로우 일산밤리단길카페점 https://blog.naver.com/soheee666/223960234321
밤리단길 디저트 맛집 카피로우 느좋 테라스 카페 https://blog.naver.com/enterside/223918465299
일산 밤리단길 | 카피로우 https://blog.naver.com/rafig0913/223881827603
✅ '카피로우 일산밤리단길카페점_review_data.csv' 파일로 저장 완료


In [45]:
df

,titles,links,contents,only_kor_contents
0,"디저트가 특히 맛있었던 일산 카페, 카피로우 일산밤리단길카페점 방문기",https://blog.naver.com/hidew777/224076747828,일산 밤리단길 쪽을 지나가다가 예전부터 눈여겨보던 ‘카피로우(CAPYLOW) 일산밤...,일산 밤리단길 쪽을 지나가다가 예전부터 눈여겨보던 카피로우 일산밤리단길카페점 에 들...
1,카피로우 일산밤리단길카페점 솔직후기,https://blog.naver.com/rhdmsdl0720/223996105928,카피로우 일산밤리단길카페점\n매일 10am-10pm\n매주 금요일 정기휴무\n밤리단...,카피로우 일산밤리단길카페점 매일 매주 금요일 정기휴무 밤리단길에 위치한 카피로우에 ...
2,카피로우 일산밤리단길카페점 내돈내산 솔직후기,https://blog.naver.com/rkguscjswo26/223891656310,저랑 남편은 케이크를 그렇게 좋아하진 않아요\n하지만 누구의 생일이면 기념하기 위해...,저랑 남편은 케이크를 그렇게 좋아하진 않아요 하지만 누구의 생일이면 기념하기 위해 ...
3,“카피로우” 일산밤리단길카페점.크렘당쥬?프랑스디저트 천사의크림일산 밤리단길카페추천,https://blog.naver.com/comma2024/223785598110,오랜만에 밤마실 나온\n자유부인!!\n✌️😉\n쓰~읍\n이 밤 공기 너무 좋구만 ㅋ...,오랜만에 밤마실 나온 자유부인 쓰 읍 이 밤 공기 너무 좋구만 간판도 아주 작은 가...
4,[경기/고양시] 일산동구 마두동 밤리단길 데이트 카페 추천 무화과 디저트 맛집 찾고...,https://blog.naver.com/vivid_07/224064134098,[경기/고양시] 일산동구 마두동 밤리단길 데이트 카페 추천 무화과 디저트 맛집 찾고...,경기 고양시 일산동구 마두동 밤리단길 데이트 카페 추천 무화과 디저트 맛집 찾고 있...
5,"일산 카페 찐맛집, 카피로우 일산밤리단길카페점",https://blog.naver.com/soheee666/223960234321,"내가 정말 정말 좋아해서,\n아는 지인, 친구 다 데려간 음료 맛집!\n2025. ...",내가 정말 정말 좋아해서 아는 지인 친구 다 데려간 음료 맛집 수 여긴 찐이다 너무...
6,밤리단길 디저트 맛집 카피로우 느좋 테라스 카페,https://blog.naver.com/enterside/223918465299,계절을 담은 디저트를\n커피와 함께 내어드립니다.\n밤리단길 디저트 맛집 카피로우 ...,계절을 담은 디저트를 커피와 함께 내어드립니다 밤리단길 디저트 맛집 카피로우 느좋 ...
7,일산 밤리단길 | 카피로우,https://blog.naver.com/rafig0913/223881827603,오랜만에 가는 일산 나들이 - 🤍\n막둥이의 최애 카페를 탐내고 있다가\n시즌 끝나...,오랜만에 가는 일산 나들이 막둥이의 최애 카페를 탐내고 있다가 시즌 끝나자마자 방문...


In [33]:
def combine_review(review_list):
    input_string = ''
    for i in review_list:
        input_string += i
    return input_string

In [ ]:
from openai import OpenAI

client = OpenAI(api_key = '')
input_string = combine_review(df['only_kor_contents'])
response = client.responses.create(
  model="gpt-5-nano",
  input=f"please summarize review  review : {input_string}" 
)

print(response.output[1].content[0].text)

다음은 해당 카피로우 일산밤리단길카페점 방문 후기의 핵심 요약입니다.

- 장소와 분위기
  - 일산 밤리단길에 위치한 카페로, 외관은 화이트 우드의 따뜻한 인테리어가 돋보이고 내부는 밝지 않은 편안한 분위기.
  - 식물과 널찍한 의자 간격으로 여유로움, 조용한 대화가 들리는 편안한 분위기. 강아지를 산책하러 온 손님도 있어 지역 카페 분위기가 느껴짐.
  - 주차는 매장 앞에 무료로 가능하고, 월요일/금요일 휴무 여부가 사람마다 다르게 언급되나, 여러 후기에서 금요일 휴무를 확인.

- 서비스와 가성비
  - 직원 친절하고 분위기가 아늑하다는 평가가 많은 편.
  - 대부분의 디저트가 “주인공”인 분위기로 커피보다 디저트가 강점으로 꼽힘.
  - 인기 탓에 품절이 잦고, 주문한 메뉴가 늦게 나오는 경우도 있음. 그래도 전반적으로 서비스는 긍정적.

- 메뉴 특징
  - 커피: 카페라떼, 땅콩크림라떼, 흑임자라떼 등 다양한 커피류가 있고 라떼의 밸런스가 좋다고 평가.
  - 디저트의 주력은 계절 한정 및 시그니처 디저트: 단호박 케이크, 가을의 선샤인(샤인 머스켓과 리코타 치즈 토핑), 흑임자 타르트, 쑥팥타르트 등 타르트류가 많고 품질이 높음.
  - 시즌에 따라 재료가 달라지는 디저트들이 많아 방문 시마다 새로운 맛을 기대할 수 있음.
  - 강력 추천 디저트: 단호박 케이크, 가을의 선샤인, 흑임자타르트, 쑥팥타르트, 무화과 디저트 등. 크렘당쥬도 자주 언급됨.
  - 디저트와 음료의 조합이 매우 좋으며, 일부 손님은 디저트를 먹으러 재방문하기도 함.
  - 사진 찍기를 좋아하는 분들을 위한 비주얼 메뉴와 독특한 잔(비커 형태 컵) 제공.

- 총평 / 추천 대상
  - 분위기, 맛, 서비스, 가격까지 전체적으로 균형 잡힌 곳으로 평가.
  - 커피도 맛있지만 디저트가 특히 강해 디저트 애호가에게 강력 추천.
  - 밤리단길에서 조용히 커피를 즐기고 싶거나 계절별 디저트를 체험하고 싶은 분에게 적합.
  - 단점으로는 인기 많아 품절/대기, 일부 메뉴의

# Langchain

ImportError: cannot import name 'ChatOpenAI' from 'langchain.chat_models' (c:\Users\hisem\Desktop\pipeline\.venv\Lib\site-packages\langchain\chat_models\__init__.py)

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model='gpt-5-nano'
)
prompt = PromptTemplate.from_template("""
다음 글을 이 탬플릿을 기반 키워드로 요약해줘:
- 대표 메뉴: ex) 소금빵, 고구마라떼, 아이스아메리카노 ...
- 분위기/경험: ex) 애견동반가능, 아늑한, 소파가 편한, 자리가 넓은, ...
- 추천 대상: ex) 반려견과 함께, 연인과 함께, 친구와 함께,...
{text}
""")

summarize_chain = prompt | model | StrOutputParser()

summary = summarize_chain.invoke({"text": input_string})
print(summary)

- 대표 메뉴: 소금빵, 커피
- 분위기/경험: 아늑한, 감성적인 인테리어, 널찍한 자리
- 추천 대상: 반려견과 함께, 연인과 함께, 친구와 함께


In [ ]:
from typing import List
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


# --- 1) 구조화된 Output 모델 ---
class ReviewExtraction(BaseModel):
    main_menu: List[str] = Field(
        ...,
        description="가게에서 많이 언급되는 대표 메뉴 키워드들 (예: 소금빵, 아메리카노, 고구마라떼)"
    )
    atmosphere: List[str] = Field(
        ...,
        description="가게 분위기, 경험, 매장 특징 키워드들 (예: 아늑한, 감성적인, 좌석이 넓은)"
    )
    recommended_for: List[str] = Field(
        ...,
        description="어떤 유형의 사람이 방문하면 좋은지 (예: 연인과 함께, 친구와 수다, 반려견과 함께)"
    )


# --- 2) LLM + 구조화 출력 준비 ---
model = ChatOpenAI(
    model_name="gpt-5-nano",
    temperature=0.2
).with_structured_output(ReviewExtraction)


# --- 3) 프롬프트 ---
prompt = PromptTemplate.from_template("""
다음 글을 아래 템플릿을 기반으로 핵심 키워드 3~4개를 추출해줘:

[템플릿]
- 대표 메뉴: ex) 소금빵, 고구마라떼, 아이스아메리카노 ...
- 분위기/경험: ex) 애견동반가능, 아늑한, 소파가 편한, 자리가 넓은, ...
- 추천 대상: ex) 반려견과 함께, 연인과 함께, 친구와 함께,...

출력 형식:
main_menu: 키워드만 리스트
atmosphere: 키워드만 리스트
recommended_for: 키워드만 리스트

리뷰 텍스트:
------------------------------------
{text}
------------------------------------
""")

# --- 4) LCEL 체인 ---
summarize_chain = prompt | model


# --- 5) 실행 예제 ---


result: ReviewExtraction = summarize_chain.invoke({"text": input_string})

print(result)
print(result.main_menu)
print(result.atmosphere)
print(result.recommended_for)


main_menu=['카페라떼', '땅콩크림라떼', '단호박케이크', '가을의 선샤인'] atmosphere=['따뜻하고 아늑한', '화이트 우드 인테리어', '조용한 분위기', '사진 잘 나오는 분위기'] recommended_for=['데이트 코스', '친구와 함께', '디저트 애호가', '반려동물과 함께(테라스)']
['카페라떼', '땅콩크림라떼', '단호박케이크', '가을의 선샤인']
['따뜻하고 아늑한', '화이트 우드 인테리어', '조용한 분위기', '사진 잘 나오는 분위기']
['데이트 코스', '친구와 함께', '디저트 애호가', '반려동물과 함께(테라스)']
